# 第4章　PyTorchを使ったニューラルネット基礎







In [2]:
!pip install torchvision==0.2.1
!pip install pillow==4.2.0

    100% |████████████████████████████████| 61kB 2.3MB/s 
    100% |████████████████████████████████| 519.5MB 37kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x59776000 @  0x7f62790a81c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 2.0MB 5.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 5.8MB 926kB/s 
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0


In [1]:
import torch
torch.__version__

'0.4.1'

In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

## PyTorchの記法

### Sequentialを使った書き方

入力と出力が１対１のネットワークを書く場合にnn.Sequentialの中に処理をまとめて書く方法があります。簡単に書ける一方で分岐など複雑なネットワークは書くことができません。


In [101]:
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )
model

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)

add_moduleを使用すると名前付きで書くことができます。

In [102]:
model = torch.nn.Sequential()
model.add_module("conv1", nn.Conv2d(1,20,5))
model.add_module("relu1", nn.ReLU())
model.add_module("conv2", nn.Conv2d(20,64,5))
model.add_module("relu2", nn.ReLU())
model

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)

PythonのOrdinalDictを使用して名前付きで書くこともできます。

In [104]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))
model

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)

# nn.Moduleを継承した書き方

Chainerでお馴染みの書き方です。最近はKerasでも似た書き方ができるようになりました。


In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
      
model = Model()
model

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
)

# nn.ModuleListを使った書き方

nn.ModuleListを使用すると層をリストにすることができます。



In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.convs = nn.ModuleList([nn.Conv2d(1, 20, 5), nn.Conv2d(20, 20, 5)])

    def forward(self, x):
        for i, l in enumerate(self.convs):
            x = l(x)
        return x
      
model = Model()
model

Model(
  (convs): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  )
)

# nn.ModuleDictを使った書き方

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.convs = nn.ModuleDict({'conv1' : nn.Conv2d(1, 20, 5), 'conv2' : nn.Conv2d(20, 20, 5)})

    def forward(self, x):
        for i, l in enumerate(self.convs):
            x = l(x)
        return x
      
model = Model()
model

Model(
  (convs): ModuleDict(
    (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  )
)